In [4]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from matplotlib.ticker import FormatStrFormatter
import math

In [63]:
# model parameters

Ka = 1e-4
c_bufs = {'-':[],'buf': []}
zs = {'K': 1,'Cl': -1,'H': 1}
Ps = {'K': 1e-6,'Cl': 1e-,'H': 1e1} #m^s-1
c = 1e-2 #Fm^-2

F = 96485 #Cmol^-1
R = 8.31 #JK^-1mol^-1
r_v = 12.5e-6 #m
T = 298 #K
A = 4*np.pi*r_v**2
vol = 4*np.pi*r_v**3
c_outs = {'K':1,'Cl':1,'H':1e-4} #Molm^-3
B = np.log(10)*20*Ka*1e-4/(2e-4)**2 #buffer capacity for my buffer. TODO: measure this!
C = c*A
eta = (zs['H']**2)*(F**2)*vol*B/(R*T*np.log(10)*C)
eta = 1e6

pHo = -1*math.log(c_outs['H'])/(math.log(10))
    
fluxes = {key:[] for key in Ps.keys()}

threshold = 0.1

c_ins = {key:[] for key in Ps.keys()}

V = []

dt = 30 #s


# Old model: seemed to be unstable

def initialise():
    
    V.append(0)
    c_ins['K'].append(0)
    c_ins['Cl'].append(0)
    c_ins['H'].append(1e-4)

    c_bufs['-'].append(1e-2) #M
    c_bufs['buf'].append(1e-2) #M
    

def dCdt(c_in,c_out,P,V, z):
    
    mu = z*F*V/(R*T)
#     print('mu = ', mu)
    if abs(mu) < threshold:
        
        return (P/(3*R))*(c_out - c_in)
    
#     print('mu = ', mu)
    
    prefactor = -mu*P/(3*R)
    
    
    return prefactor*(c_out - c_in*np.exp(-mu))/(np.exp(-mu)-1)


def dV(dCs,vol,eta,C,zs):
    
    # args:
    # dCs: dictionary of ion molar fluxes
    # vol: volume of GUV
    # eta: dimensionless constant. the voltage change due to one unit of pH change
    # C: capacitance of GUV membrane
    
    prefactor = 1/(C*(1+eta))
    
    total_Q_flux =0
    for key in dCs.keys():
        if key not in ['K','Cl']:
            continue
            
            
        total_Q_flux += zs[key]*F*vol*dCs[key][-1]
        
#     print('total charge flux: ',total_Q_flux)
    return prefactor*total_Q_flux


"""
def adjustcH(cH_0,Ka,c_buf,dcH):
    
    return c_buf*dcH/(Ka+cH_0+c_buf)

def calcV(Q,C,A):
#     print(Q)
    return Q/(C*A)


def Q_in(c_ins,vol):
    
    Q = 0 
    for key in c_ins.keys():
#         print(c_ins[key])
        Q += vol*F*zs[key]*c_ins[key][-1]
        
#     print('Q = ',Q)
    return Q
"""    

def cH(V,pHo):
    

    
    neg_pH = F*V[-1]/(R*T*math.log(10))-pHo
#     print('pH inside GUV: ',-neg_pH)
    return 10**neg_pH

def step_forward(c_ins,V,c_bufs):
    
    # calculate fluxes and increment the internal concentrations
    
    
    for key in c_ins.keys():
        if key not in ['K','Cl']:
            continue
            
        dC = dt*dCdt(c_ins[key][-1],c_outs[key],Ps[key],V[-1],zs[key])
        fluxes[key].append(dC)
        
        c_ins[key].append(c_ins[key][-1]+dC)
        
        
    # adjust the H+ concentration due to the buffer
    
#     dcH = c_ins['H'][-2] - c_ins['H'][-1]
    
#     c_ins['H'][-1] += adjustcH(c_ins['H'][-1],Ka,c_bufs['-'][-1],dcH)
    
    
#     Q = Q_in(c_ins,vol)
#     print(Q)
    V.append(V[-1]+dV(fluxes,vol,eta,C,zs))
    c_ins['H'].append(cH(V,pHo))


In [44]:
eta

1000000.0

In [64]:
#define times

ts = np.arange(0,500000)

# initialise


initialise()


# start time loop
counter = 0
for t in ts:
    if not t % 1000 and t > 0:
        
        
        V = V[:counter]+[V[-1]]
#         print(V)
        for key in c_ins.keys():
            c_ins[key] = c_ins[key][:counter]+[c_ins[key][-1]]
            
    counter +=1
    step_forward(c_ins,V,c_bufs)

In [65]:
%matplotlib qt
fig,axs = plt.subplots(1,4)


for i,key in enumerate(c_ins.keys()):

        
    axs[i].plot(c_ins[key][:-1])
    
    axs[i].xaxis.set_major_formatter(FormatStrFormatter('%.2f'))
    
    axs[i].set_title(key,fontsize = 18)
    axs[i].set_xlabel('T [mins]')
#     axs[i].set_xticks([0,1000,2000])
#     axs[i].set_xticklabels([0,1000//60,2000//60],fontsize = 15)
#     axs[i].set_xlim([-50,2050])

    
# axs[0].set_ylim([0,1])
# axs[1].set_ylim([0,1])
axs[3].plot(V[:-1])
axs[3].set_title('V_m')
# axs[3].set_xticks([0,1000,2000])
# axs[3].set_xticklabels([0,1000//60,2000//60],fontsize = 15)
# axs[3].set_xlim([-50,2050])
# axs[3].set_ylim([0,1])

Text(0.5, 1.0, 'V_m')

In [61]:
c_ins['H']

[0.0001,
 0.00010004235413724889,
 0.00010008472621305701,
 0.00010012711623502205,
 0.00010016952421074498,
 0.00010021195014782975,
 0.00010025439405388346,
 0.00010029685593651678,
 0.0001003393358033432,
 0.00010038183366197971,
 0.00010042434952004646,
 0.0001004668833851668,
 0.00010050943526496731,
 0.00010055200516707781,
 0.00010059459309913135,
 0.00010063719906876422,
 0.00010067982308361593,
 0.00010072246515132913,
 0.00010076512527955001,
 0.0001008078034759276,
 0.00010085049974811446,
 0.0001008932141037664,
 0.00010093594655054217,
 0.00010097869709610406,
 0.00010102146574811752,
 0.00010106425251425123,
 0.00010110705740217711,
 0.00010114988041957026,
 0.0001011927215741093,
 0.00010123558087347583,
 0.00010127845832535468,
 0.00010132135393743424,
 0.00010136426771740577,
 0.00010140719967296406,
 0.00010145014981180708,
 0.00010149311814163608,
 0.00010153610467015553,
 0.00010157910940507317,
 0.00010162213235410005,
 0.00010166517352495041,
 0.000101708232925341

# %matplotlib qt
fig,axs = plt.subplots(1,4)

for i,key in enumerate(fluxes.keys()):
    
    axs[i].plot(fluxes[key])
    
    axs[i].set_title(key)

In [173]:
vol*F*1/(C*A)

1.2060625000000003